In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
from datetime import datetime, date
from fuzzywuzzy import fuzz
import random

load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
refresh_token = os.getenv('REFRESH_TOKEN')
OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')
current_date = datetime.now().date()
cutoff_in_days=21

In [2]:
events = pd.read_csv('raw_events.csv').drop_duplicates(['raw_event_name', 'raw_date_text'])

auth_manager = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri="http://127.0.0.1:8080",
    scope="playlist-modify-public playlist-modify-private"
)
token_info = auth_manager.refresh_access_token(refresh_token)
sp = spotipy.Spotify(auth=token_info['access_token'])
print(events.columns)

Index(['venue_id', 'raw_event_name', 'raw_date_text', 'genres', 'is_cancelled',
       'parsed_date'],
      dtype='object')


In [3]:
def validate_artist(events, raw_event_name):
    results = sp.search(q=f'artist:{raw_event_name}', type='artist', limit=3)
    # if the artist exists in spotify
    if results['artists']['items']:
        name = results['artists']['items'][0]['name']
        # and the name is a match
        if fuzz.ratio(raw_event_name.lower(), name.lower()) > 90:
            genres = results['artists']['items'][0]['genres']
            id = results['artists']['items'][0]['id']
            popularity = results['artists']['items'][0]['popularity']
            # add them to the df
            events.loc[events['raw_event_name'] == raw_event_name, 'genres'] = ', '.join(genres)
            events.loc[events['raw_event_name'] == raw_event_name, 'spotify_artist_name'] = name
            events.loc[events['raw_event_name'] == raw_event_name, 'spotify_artist_id'] = id
            events.loc[events['raw_event_name'] == raw_event_name, 'artist_popularity'] = popularity
        else:
            print(raw_event_name, 'name did not match spotify -->', name)
    else:
        print(raw_event_name,'could not be found on spotify')

In [4]:
raw_event_names = events['raw_event_name'].values

for event_name in raw_event_names:
    validate_artist(events, event_name)
events

Neck of the Woods SF Open Mic Wednesdays could not be found on spotify
Fox Lake: The New World Heat Tour could not be found on spotify
What's Next could not be found on spotify
Culture Over Everythin' could not be found on spotify
Sybyr/ Yvncc/ Bruhmanegod/ Melodrama/ $av could not be found on spotify
Wrath of Logarius/ Imperialist/ The Dark Alamorte/ Post Heaven/ Sicarius could not be found on spotify
Dontcryformeimalreadydead/ Bucky/ Let Them Eat Horses/ Boy Talk/ The Most Sad could not be found on spotify
Salsa Crazy Mondays could not be found on spotify
Karaoke Tuesday could not be found on spotify
Two Piece/ Chain Gang/ Cycle of Abuse/ Shortiez/ Bog Body could not be found on spotify
Neck of the Woods SF Open Mic Wednesdays could not be found on spotify
Professa Gabel & The Top Chefs could not be found on spotify
Salsa Crazy Mondays could not be found on spotify
Popstar Booty Camp Dance Performance could not be found on spotify
Karaoke Tuesday could not be found on spotify
Neck of

,venue_id,raw_event_name,raw_date_text,genres,is_cancelled,parsed_date,spotify_artist_name,spotify_artist_id,artist_popularity
0,12,Neck of the Woods SF Open Mic Wednesdays,"Wed, Oct 15",NaN,False,2025-10-15,NaN,NaN,NaN
1,12,Fox Lake: The New World Heat Tour,"Thu, Oct 16",NaN,False,2025-10-16,NaN,NaN,NaN
2,12,What's Next,"Fri, Oct 17",NaN,False,2025-10-17,NaN,NaN,NaN
3,12,Easha,"Fri, Oct 17",bedroom pop,False,2025-10-17,EASHA,1lYNHBChDDWC233o6ribbt,47.0
4,12,Culture Over Everythin',"Sat, Oct 18",NaN,False,2025-10-18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
371,14,Northern Flight: A Bluebird Music Festival Event,"Friday, October 10 @ 7:00 pm",NaN,False,2025-10-10,NaN,NaN,NaN
372,14,**Moved to Friday**Northern Flight: (night 2),"Saturday, October 11 @ 7:00 pm",NaN,False,2025-10-11,NaN,NaN,NaN
373,14,An Evening with Grateful Shred,"Saturday, October 18 @ 8:00 pm",NaN,False,2025-10-18,NaN,NaN,NaN
374,14,The Millennium Tour presented by Fool House,"Friday, October 24 @ 7:30 pm",NaN,False,2025-10-24,NaN,NaN,NaN


In [5]:
print(f'missing {events["spotify_artist_name"].isna().sum()} out of {len(raw_event_names)}')
missing_artists = events[events['spotify_artist_name'].isna()]['raw_event_name']

missing 106 out of 308


In [6]:
from openai import OpenAI
import json
import os

def parse_missed_artists(artist_list):
    """
    Use OpenAI to filter out non-artists (events, tribute acts, etc.)
    Returns only actual musical artists/bands

    right now we're sending all the artists from all the venues, but it might be better to send all the artists from one venue at a time.
    this way the LLM can see consistent formatting.
    """
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    # Remove duplicates first
    unique_artists = list(set(artist_list))
    
    prompt = f"""You are analyzing a list of names scraped from music venue websites. 
Some are actual musical artists/bands, and some are event names or non-musical events.

Your task: Extract and clean all PERFORMING MUSICAL ACT names.

STEP 1 - IDENTIFY if the entry contains performing musical acts:
   KEEP: Musicians, bands, DJs, tribute acts - anyone who performs music
   FILTER OUT: Event series (EMO NITE, Nerd Nite), private events, non-music events

STEP 2 - CLEAN the artist names:
   - Remove promotional text: "An Evening with", "Presented by", "Live in Concert"
   - Remove tour names: "- World Tour", "2024 Tour"  
   - Remove location info: "at [venue] - [city]" (but ONLY if it's part of an artist name, not if the whole thing is an event)
   - Remove "feat.", "featuring", "with special guest" and similar

STEP 3 - SPLIT multi-artist bills:
   - "Artist A, Artist B" → ["Artist A", "Artist B"]
   - "Artist A & Artist B" → ["Artist A", "Artist B"]  
   - "Artist A + Artist B" → ["Artist A", "Artist B"]
   - BUT preserve band names with natural "&" or "," (like "The Army, The Navy" or "Simon & Garfunkel" or "Andy Frasco and the U.N.")

Examples:
- "Legend Zeppelin" → ["Legend Zeppelin"]
- "EMO NITE at Rickshaw Stop - San Francisco, CA" → [] (entire thing is an event brand, filter out)
- "Nerd Nite SF" → [] (event series, filter out)
- "Nora Brown, Stephanie Coleman" → ["Nora Brown", "Stephanie Coleman"] (two artists)
- "Josh Ritter and the Royal City Band" → ["Josh Ritter and the Royal City Band"] (one act)
- "Pete Yorn – You and Me Solo Acoustic" → ["Pete Yorn"] (remove tour name)
- "Private Event" → [] (filter out)
- "Khalil, Amal, TRAVIE BOBBITO, KING MOST, BELLA D. & FRIENDS" → ["Khalil", "Amal", "TRAVIE BOBBITO", "KING MOST", "BELLA D."] (split multi-artist showcase)

Return a JSON OBJECT (not array) where:
- Keys are the original raw names from the input
- Values are arrays of cleaned artist names (empty array if filtered out)

Example output format:
{{
  "EMO NITE at Rickshaw Stop": [],
  "Nora Brown, Stephanie Coleman": ["Nora Brown", "Stephanie Coleman"],
  "XANA": ["XANA"]
}}

Names to evaluate:
{json.dumps(unique_artists, indent=2)}

Respond with ONLY the JSON object, no other text."""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a music industry expert who can distinguish between artist names and event names."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1 
    )
    
    # Parse the response
    result_text = response.choices[0].message.content.strip()
    
    # Remove markdown code blocks if present
    if result_text.startswith('```'):
        result_text = result_text.split('```')[1]
        if result_text.startswith('json'):
            result_text = result_text[4:]
    
    filtered_mapping = json.loads(result_text)
    
    # Count stats
    total_filtered = sum(len(artists) for artists in filtered_mapping.values())
    total_removed = sum(1 for artists in filtered_mapping.values() if len(artists) == 0)
    
    print(f"📊 Input: {len(unique_artists)} unique raw names")
    print(f"✅ Output: {total_filtered} cleaned artist names")
    print(f"🗑️  Filtered out: {total_removed} non-artists")
    
    return filtered_mapping

In [7]:
event_artist_map = parse_missed_artists(missing_artists.values)


📊 Input: 96 unique raw names
✅ Output: 111 cleaned artist names
🗑️  Filtered out: 31 non-artists


In [9]:
new_rows = []

for (raw_event_name, cleaned_artists) in event_artist_map.items():
    # Get the original event data to copy venue_id, date info, etc.
    original_event = events[events['raw_event_name'] == raw_event_name].iloc[0]
    
    for artist in cleaned_artists:
        # Get the artist from spotify
        results = sp.search(q=f'artist:{artist}', type='artist', limit=3)
        
        # If you find it on spotify
        if results['artists']['items']:
            name = results['artists']['items'][0]['name']
            # And the names match
            if fuzz.ratio(artist.lower(), name.lower()) > 80:
                genres = results['artists']['items'][0]['genres']
                artist_id = results['artists']['items'][0]['id']
                popularity = results['artists']['items'][0]['popularity']
                
                # Create new row
                new_row = {
                    'venue_id': original_event['venue_id'],
                    'raw_event_name': raw_event_name,
                    'raw_date_text': original_event['raw_date_text'],
                    'parsed_date': original_event['parsed_date'],
                    'spotify_artist_id': artist_id,
                    'spotify_artist_name': name,
                    'artist_popularity': popularity,
                    'genres': genres,
                    'is_cancelled': False
                }
                new_rows.append(new_row)
            else:
                print(f"{artist} name did not match spotify --> {name}")
        else:
            print(f"{artist} could not be found on spotify")

# Append new rows to existing DataFrame
missed_events_df = pd.DataFrame(new_rows)
events = pd.concat([events, missed_events_df], ignore_index=True).drop_duplicates(['raw_event_name', 'raw_date_text'])
events

ORIHUELA name did not match spotify --> Orihuela M.S.S.
Tape B name did not match spotify --> The New Basement Tapes
Hobo Johnson and The Lovemakers could not be found on spotify
Yīn Yīn name did not match spotify --> Yin Yin
AZ/DZ name did not match spotify --> Dzidzor Azaglo
Lebo name did not match spotify --> Lebo M.
Bo Carper could not be found on spotify
Electric Feels could not be found on spotify
Rumours name did not match spotify --> Yea-Ming and The Rumours
Luke Hahn-Zollo could not be found on spotify
UMI name did not match spotify --> Artists United Against Apartheid
Super Diamond name did not match spotify --> Da Superhero & DIAMONDS
Josh Ritter and the Royal City Band could not be found on spotify
Jack's Mannequin name did not match spotify --> Peter Jackson ft. Die Mannequin and Shadow Bloc
HEALTH name did not match spotify --> Gut Health
Say What?! name did not match spotify --> What Laura Says
KAFANA BALKAN could not be found on spotify
Mo Lowda and The Humble could not

,venue_id,raw_event_name,raw_date_text,genres,is_cancelled,parsed_date,spotify_artist_name,spotify_artist_id,artist_popularity
0,12,Neck of the Woods SF Open Mic Wednesdays,"Wed, Oct 15",NaN,False,2025-10-15,NaN,NaN,NaN
1,12,Fox Lake: The New World Heat Tour,"Thu, Oct 16",NaN,False,2025-10-16,NaN,NaN,NaN
2,12,What's Next,"Fri, Oct 17",NaN,False,2025-10-17,NaN,NaN,NaN
3,12,Easha,"Fri, Oct 17",bedroom pop,False,2025-10-17,EASHA,1lYNHBChDDWC233o6ribbt,47.0
4,12,Culture Over Everythin',"Sat, Oct 18",NaN,False,2025-10-18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
374,12,Wrath of Logarius/ Imperialist/ The Dark Alamo...,"Sun, Oct 19","[deathcore, death metal]",False,2025-10-19,The Dark Alamorte,3Sr4CCs7XORgQd5x28Zntg,5.0
375,12,Wrath of Logarius/ Imperialist/ The Dark Alamo...,"Sun, Oct 19",[],False,2025-10-19,Post Heaven,2MAGC2Ja0QAYo9GTq3rLIM,6.0
376,15,Graham Good & The Painters + Spring Polar Plun...,"Sat, Mar 07, 2026",[],False,2026-03-07,Graham Good,68cQzWn73nEovfjcGt3ktw,22.0
377,9,"Patterson Hood, Craig Finn",Mon Nov 10,"[alt country, americana]",False,2025-11-10,Patterson Hood,5JnjssuFBg104paXN0mKKn,33.0


In [16]:
events = events.drop_duplicates(['raw_event_name', 'raw_date_text'])
events[~events['spotify_artist_id'].isna()]

,venue_id,raw_event_name,raw_date_text,genres,is_cancelled,parsed_date,spotify_artist_name,spotify_artist_id,artist_popularity
3,12,Easha,"Fri, Oct 17",bedroom pop,False,2025-10-17,EASHA,1lYNHBChDDWC233o6ribbt,47.0
12,12,Outside World,"Thu, Oct 23","jazz fusion, nu jazz",False,2025-10-23,Outside World,5xWSapntbvFDBkJ2y37CKf,10.0
18,9,XANA,Wed Oct 1,,False,2025-10-01,Xana,5Ui0TOcqUVmXQORMMkMaD3,54.0
19,9,MADISON MCFERRIN,Fri Oct 3,alternative r&b,False,2025-10-03,Madison McFerrin,02zPEtdzUWnPToEVLRiQ7e,63.0
21,9,UPCHUCK,Mon Oct 6,egg punk,False,2025-10-06,Upchuck,7mhCgHESQzEKKyWshkMTwW,38.0
...,...,...,...,...,...,...,...,...,...
295,10,Rich Brian,"Fri, Mar 13, 2026",indonesian hip hop,False,2026-03-13,Rich Brian,2IDLDx25HU1nQMKde4n61a,70.0
296,10,Ashnikko,"Sat, Mar 21, 2026",,False,2026-03-21,Ashnikko,3PyJHH2wyfQK3WZrk9rpmP,73.0
298,10,Peter Hook & The Light,"Mon, Sep 21, 2026","madchester, post-punk",False,2026-09-21,Peter Hook and The Light,5u3A3inCFA998oODFCRk6S,27.0
299,14,YOURS TRULY,"Thursday, October 2 @ 7:30 pm",pop punk,False,2025-10-02,Yours Truly,76NpRNEWMaNdOudixwOPRo,44.0


In [ ]:
def add_songs_from_artist(sp, artist_id, playlist_id, n=3, country_code='US'):
    top_tracks = sp.artist_top_tracks(artist_id)['tracks']
    n_tracks_to_add = min(n, len(top_tracks))
    i_to_add = random.sample(range(len(top_tracks)), n_tracks_to_add)
    tracks_to_add = [top_tracks[i] for i in i_to_add]

    track_uris = [track['uri'] for track in tracks_to_add]
    sp.playlist_add_items(playlist_id, track_uris)
    print(f"added {len(track_uris)} songs")

In [17]:
conn = psycopg2.connect(os.getenv('DATABASE_URL_UNPOOLED'))
cur = conn.cursor()


alter_table = """ALTER TABLE artist_events 
ADD CONSTRAINT unique_artist_event 
UNIQUE (venue_id, spotify_artist_name, event_date);"""


# # Create artist_events table
# create_table_query = """
# CREATE TABLE artist_events (
#     id SERIAL PRIMARY KEY,
#     venue_id INT REFERENCES venues(venue_id),
#     raw_event_name VARCHAR(500),
#     raw_date_text VARCHAR(100),
#     genre VARCHAR(100),
#     is_cancelled BOOLEAN DEFAULT FALSE,
#     spotify_artist_name VARCHAR(255),
#     spotify_artist_id VARCHAR(50),
#     artist_popularity INT,
#     event_date DATE,
#     scraped_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
# );
# """

cur.execute(alter_table)

# # Create indexes for better query performance
# cur.execute("CREATE INDEX idx_artist_events_venue_id ON artist_events(venue_id);")
# cur.execute("CREATE INDEX idx_artist_events_event_date ON artist_events(event_date);")
# cur.execute("CREATE INDEX idx_artist_events_spotify_id ON artist_events(spotify_artist_id);")

conn.commit()


In [19]:
import psycopg2.extras
conn = psycopg2.connect(os.getenv('DATABASE_URL_UNPOOLED'))
cur = conn.cursor()

# Upsert query - updates if conflict on unique constraint
insert_query = """
INSERT INTO artist_events (
    venue_id,
    raw_event_name,
    raw_date_text,
    genre,
    is_cancelled,
    spotify_artist_name,
    spotify_artist_id,
    artist_popularity,
    event_date
) VALUES %s
ON CONFLICT (venue_id, spotify_artist_name, event_date) 
DO UPDATE SET
    raw_event_name = EXCLUDED.raw_event_name,
    raw_date_text = EXCLUDED.raw_date_text,
    genre = EXCLUDED.genre,
    is_cancelled = EXCLUDED.is_cancelled,
    artist_popularity = EXCLUDED.artist_popularity,
    scraped_at = CURRENT_TIMESTAMP
"""

# Convert DataFrame to list of tuples
data_tuples = []
for _, row in events.iterrows():
    data_tuple = (
        int(row['venue_id']),
        row['raw_event_name'],
        row['raw_date_text'],
        ','.join(row['genres']) if (isinstance(row['genres'], list) and len(row['genres']) > 0) else None,
        bool(row['is_cancelled']),
        row['spotify_artist_name'] if pd.notna(row['spotify_artist_name']) else None,
        row['spotify_artist_id'] if pd.notna(row['spotify_artist_id']) else None,
        int(row['artist_popularity']) if pd.notna(row['artist_popularity']) else None,
        row['parsed_date'] if pd.notna(row['parsed_date']) else None
    )
    data_tuples.append(data_tuple)

# Execute bulk upsert
psycopg2.extras.execute_values(
    cur,
    insert_query,
    data_tuples,
    template=None,
    page_size=100
)

conn.commit()
print(f"Upserted {len(data_tuples)} rows into artist_events table")

Upserted 308 rows into artist_events table
